In [34]:
import pandas as pd
import numpy as np
import scipy
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle



import gensim
from gensim import corpora, models, matutils
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction import _stop_words
from nltk.stem import WordNetLemmatizer


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [35]:
df=pd.read_csv('stock_data_nlp.csv',encoding= 'unicode_escape')
df.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite


In [36]:
df.iloc[0,4]

"Hughes' instant hit buoys Blues"

In [37]:
df.shape

(4262, 27)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4262 entries, 0 to 4261
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    4262 non-null   object
 1   Label   4214 non-null   object
 2   Top1    4210 non-null   object
 3   Top2    4202 non-null   object
 4   Top3    4192 non-null   object
 5   Top4    4169 non-null   object
 6   Top5    4158 non-null   object
 7   Top6    4151 non-null   object
 8   Top7    4144 non-null   object
 9   Top8    4135 non-null   object
 10  Top9    4127 non-null   object
 11  Top10   4118 non-null   object
 12  Top11   4106 non-null   object
 13  Top12   4100 non-null   object
 14  Top13   4091 non-null   object
 15  Top14   4083 non-null   object
 16  Top15   4075 non-null   object
 17  Top16   4071 non-null   object
 18  Top17   4064 non-null   object
 19  Top18   4054 non-null   object
 20  Top19   4049 non-null   object
 21  Top20   4040 non-null   object
 22  Top21   4030 non-null   

In [39]:
df.Label.value_counts()

1                                                                                                                                                                                                                                                                                                     2166
0                                                                                                                                                                                                                                                                                                     1935
b'FBI arrests 10 Russian Spies in U.S.'                                                                                                                                                                                                                                                                  1
b'PHILIPPINES Key witness in the Maguindanao massacre case killed'                                     

In [40]:
#remove the date column
#the stock close trend is directly affected by the news from the same day
df=df.drop(['Date'],axis=1)

In [41]:
df.head()

,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar's debut double,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,Hopkins 'furious' at Foster's lack of Hannibal...,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,England's decade of disasters,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,Alan Parker - part two,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft


In [42]:
# combine all 25 news of the same day into a long string for doc2vec and vectorize
combine_news=[]

for index,row in df.iterrows():
    combine_news.append(' '.join(str(x) for x in row[1:]))

df['news']=pd.DataFrame(combine_news)
    

In [43]:
df.loc[1,'news']

"Scorecard The best lake scene Leader: German sleaze inquiry Cheerio, boyo The main recommendations Has Cubie killed fees? Has Cubie killed fees? Has Cubie killed fees? Hopkins 'furious' at Foster's lack of Hannibal appetite Has Cubie killed fees? A tale of two tails I say what I like and I like what I say Elbows, Eyes and Nipples Task force to assess risk of asteroid collision How I found myself at last On the critical list The timing of their lives Dear doctor Irish court halts IRA man's extradition to Northern Ireland Burundi peace initiative fades after rebels reject Mandela as mediator PE points the way forward to the ECB Campaigners keep up pressure on Nazi war crimes suspect Jane Ratcliffe Yet more things you wouldn't know without the movies Millennium bug fails to bite"

In [44]:
# Find all the 2/3 character abbrevations from the string corpus. Convert them to full form. 
# Then we can remove the punctuations without worrying losing the meaning of the abbrevation words.

# combine all the news into a very long string
long_news_str = '   '.join(df.news)

# find all the abbrevations of 2 and 3 characters
def find_abbr(text):
    abbr = []
    for i in re.finditer(r"([A-Za-z]+| )([A-Za-z]\.){2,}", text):
        abbr.append(i.group())
    df_abbr = pd.Series(abbr)
    return df_abbr.unique()

find_abbr(long_news_str)

array([' A.I.', ' M.A.N.D.Y.', ' U.N.', ' U.S.', ' U.K.', ' S.A.',
       ' U.S.C.', ' D.C.', ' N.J.', ' i.e.', ' P.I.', ' A.N.C.', ' a.m.',
       ' A.K.A.', ' P.R.', ' R.I.', ' E.U.', ' H.I.V.', ' I.H.T.',
       ' B.C.', ' J.P.', ' N.S.', 'crimese.g.', ' C.I.A.', ' p.m.',
       'Ph.D.', ' N.Y.', ' U.A.E.', 'sq.m.', ' I.M.F.', ' y.o.', ' i.a.',
       ' I.D.', ' M.A.', ' H.W.', ' O.K.', ' N.K.', ' B.S.', ' A.T.M.',
       ' W.H.O.', ' N.S.A.', ' P.M.', ' F.B.I.', ' P.E.I.', ' a.k.a.',
       ' S.E.', ' A.D.', ' T.B.', ' J.K.', ' L.G.B.T.'], dtype=object)

In [45]:
# get ready stop words for the tokenization function below
mywords = ['breaking','whilst', 'say', 'says', 'today','yesterday', 'news', 'tomorrow','iii', 'ii', 'like', 'ha',]
final_stop = stopwords.words('english') + mywords

In [46]:
def text_to_wordlist(text, remove_stop_words=True, lemma_words=True):
    
    ''' Clean each document into a list of words:
    1. convert abbrevations to full words
    2. tokenize the text
    3. remove non-alphabetic characters and one-letter words, including numbers and punctuations
    4. remove stop words
    '''
    # clean the text, convert only the abbrs that are meaningful
    text = re.sub(r" A.T.M. ", " Automated Teller Machine ", text)
    text = re.sub(r" C.I.A. ", " Central Intelligence Agency ", text)
    text = re.sub(r" D.C. ", " District of columbia ", text)
    text = re.sub(r" E.U. ", " Europian Union ", text)
    text = re.sub(r" F.B.I. ", " Federal Bureau of Investigation ", text)
    text = re.sub(r" H.I.V. ", " Human immunodeficiency virus ", text)
    text = re.sub(r" I.H.T. ", " inheritance tax ", text)
    text = re.sub(r" I.M.F. ", " International Monetary Fund ", text)
    text = re.sub(r" I.D. ", " identification ", text)
    text = re.sub(r" L.G.B.T. ", " minority ", text)
    text = re.sub(r" M.A. ", " Massachusetts ", text)
    text = re.sub(r" N.J. ", " new jersey ", text)
    text = re.sub(r" N.K. ", " north korea ", text)
    text = re.sub(r" N.S.A. ", " National Security Agency ", text)
    text = re.sub(r" N.Y. ", " new york ", text)
    text = re.sub(r" P.E.I. ", " Prince Edward Island ", text)
    text = re.sub(r" P.M. ", " prime minister ", text)
    text = re.sub(r" P.R.C ", " china ", text)
    text = re.sub(r" S.A. ", " south africa ", text)
    text = re.sub(r" R.I. ", " Rhode Island ", text)
    text = re.sub(r" U.A.E. ", " United Arab Emirates ", text)
    text = re.sub(r" U.K. ", " england ", text)
    text = re.sub(r" U.N. ", " new jersey ", text)
    text = re.sub(r" U.S. ", " america ", text)
    text = re.sub(r" U.S.C. ", " university of south california ", text)
    text = re.sub(r" W.H.O ", " world health organization ", text)
    text = re.sub(r" a.m. ", " morning ", text)
    text = re.sub(r" p.m. ", " afternoon ", text)
    text = re.sub(r" Ph.D. ", " doctor of philosophy ", text)
    text = re.sub(r" sq.m. ", " square meter ", text)
    
    # Tokenize the string into word tokens
    tokens = word_tokenize(text)
    
    # further clean the tokens: split toekns like "b'Russia" which still have punctuations in the token
    ls = []
    for word in tokens:
        if "'" in word:
            ls = ls + word.split("'")
    tokens = tokens + ls
    
    # Optionally, shorten words to their stems
    if lemma_words:
        tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens]
    
    #Remove one letter tokens & non-alphabetic tokens, such as punctuation, then lower the tokens
    tokens = [word.lower() for word in tokens if (word.isalpha() and len(word)>1)]

    # remove stop words  
    if remove_stop_words:
        tokens = [word for word in tokens if word not in final_stop]
               
    return tokens

In [47]:
# convert each document to list of words
df.news = df.news.apply(lambda x: text_to_wordlist(x))

In [48]:
df.news[1]

['scorecard',
 'best',
 'lake',
 'scene',
 'leader',
 'german',
 'sleaze',
 'inquiry',
 'cheerio',
 'boyo',
 'main',
 'recommendation',
 'cubie',
 'killed',
 'fee',
 'cubie',
 'killed',
 'fee',
 'cubie',
 'killed',
 'fee',
 'hopkins',
 'foster',
 'lack',
 'hannibal',
 'appetite',
 'cubie',
 'killed',
 'fee',
 'tale',
 'two',
 'tail',
 'elbows',
 'eyes',
 'nipples',
 'task',
 'force',
 'ass',
 'risk',
 'asteroid',
 'collision',
 'found',
 'last',
 'critical',
 'list',
 'timing',
 'life',
 'dear',
 'doctor',
 'irish',
 'court',
 'halt',
 'ira',
 'man',
 'extradition',
 'northern',
 'ireland',
 'burundi',
 'peace',
 'initiative',
 'fade',
 'rebel',
 'reject',
 'mandela',
 'mediator',
 'pe',
 'point',
 'way',
 'forward',
 'ecb',
 'campaigners',
 'keep',
 'pressure',
 'nazi',
 'war',
 'crime',
 'suspect',
 'jane',
 'ratcliffe',
 'yet',
 'thing',
 'would',
 'know',
 'without',
 'movie',
 'millennium',
 'bug',
 'fails',
 'bite',
 'furious']

In [49]:
#save it in a binary format for future use
with open('news_wordlists.pkl', 'wb') as f:
    pickle.dump(df, f)


## Word2Vec

In [50]:
import gensim.downloader as api

In [51]:
corpus=api.load('text8')

In [52]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(corpus)

In [53]:
economy=model.wv.get_vector('south')
print(economy)

[-2.5987175  -0.80587035 -0.95448065 -0.17584535  1.6402586   0.33239534
  1.1494983   3.3212457   1.9735862  -0.4362271   3.8091543   1.1619183
  1.3889115   1.5659521   1.230937    0.32075545 -1.8033501   0.90502316
 -1.4257812  -0.501368    2.5661654  -1.4362558   1.879667    3.1524096
 -0.02955896  3.4237106  -1.6533194   1.4799217   0.04200808  3.9440286
  0.8556895  -1.6349654   2.6019468   0.5151551  -2.0577474  -3.57819
  1.4142251   0.33669686 -0.33714932  0.9488706  -0.15732737 -0.03520306
  3.4817674   0.8645772   1.0778284  -1.4529687  -0.52199686 -0.19434701
  1.4835919   0.70183843  0.8874306  -0.48945007  2.863426    2.3381586
 -0.21219166  0.459196   -0.16720808  1.2715899   2.179649    1.1103697
 -1.6399026  -1.2264462   0.9928434   0.87965554  0.96227074  2.5315156
 -0.17717327 -1.4707731  -0.5855103  -0.5886246  -0.9540854   1.0524472
 -0.23737822 -1.9311324  -0.5570113   0.7065351  -1.2808826   1.2223835
 -1.3368782   2.60954    -1.8873404  -1.5932667   0.88349825  

In [54]:






def doc2vec(model,wordlist):
    vector_list=[model.wv.get_vector(word) for word in wordlist if word in model.wv]
    doc_vector=np.mean(vector_list,axis=0)
    return doc_vector


In [55]:
# convert each document (list of words) to a document vecter, then save into a list of doc_vec
x_doc = [doc2vec(model, doc) for doc in df.news]  
X_doc= pd.Series(x_doc, name = 'doc_vec') # list to df

In [56]:
X_doc.shape

(4262,)

In [57]:
# a new dataframe containing only label, news, doc_vec
newdf = pd.concat([df.Label,df.news,X_doc], axis = 1)

# convert each row in the news column from a list of tokens to a string
tokenstrlist = []
for tokenlist in  df.news:
    tokenstr = ' '.join(tokenlist)
    tokenstrlist.append(tokenstr)

# add the string format new as clolumn 'news_str' to the data frame
newdf['news_str'] = pd.Series(tokenstrlist)

# the final data frame is ready for modeling
newdf.head()



,Label,news,doc_vec,news_str
0,0,"[operation, extract, leaked, report, scorecard...","[-0.22363636, 0.043663323, 0.40945154, -0.0905...",operation extract leaked report scorecard hugh...
1,0,"[scorecard, best, lake, scene, leader, german,...","[-0.30296353, -0.33825478, 0.24209553, -0.0382...",scorecard best lake scene leader german sleaze...
2,0,"[coventry, caught, counter, flo, united, rival...","[-0.3682509, 0.26010576, 0.38839033, 0.1747563...",coventry caught counter flo united rival road ...
3,1,"[pilgrim, know, progress, thatcher, facing, ba...","[-0.3820548, -0.32600626, 0.36784732, -0.03142...",pilgrim know progress thatcher facing ban mcil...
4,1,"[hitches, horlocks, beckham, united, survive, ...","[-0.30124283, -0.22662897, 0.33770743, 0.15407...",hitches horlocks beckham united survive breast...


In [58]:
newdf = newdf[newdf['Label'].isin(['0', '1'])]

In [59]:
newdf['Label'].dtype

dtype('O')

In [65]:

newdf['Label'] = newdf['Label'].astype(int)


In [66]:
with open('label_news_docvec_newsstr.pkl', 'wb') as f:
    pickle.dump(newdf, f)

### Finding the best baseline Model
##### every dataset is different
We dont know which model works best for our data until we try. 

In [67]:
from sklearn.metrics import classification_report


In [68]:
with open('label_news_docvec_newsstr.pkl', 'rb') as r:
    df = pickle.load(r)
df.head()

,Label,news,doc_vec,news_str
0,0,"[operation, extract, leaked, report, scorecard...","[-0.22363636, 0.043663323, 0.40945154, -0.0905...",operation extract leaked report scorecard hugh...
1,0,"[scorecard, best, lake, scene, leader, german,...","[-0.30296353, -0.33825478, 0.24209553, -0.0382...",scorecard best lake scene leader german sleaze...
2,0,"[coventry, caught, counter, flo, united, rival...","[-0.3682509, 0.26010576, 0.38839033, 0.1747563...",coventry caught counter flo united rival road ...
3,1,"[pilgrim, know, progress, thatcher, facing, ba...","[-0.3820548, -0.32600626, 0.36784732, -0.03142...",pilgrim know progress thatcher facing ban mcil...
4,1,"[hitches, horlocks, beckham, united, survive, ...","[-0.30124283, -0.22662897, 0.33770743, 0.15407...",hitches horlocks beckham united survive breast...


In [69]:
df.iloc[0,2][:20]

array([-0.22363636,  0.04366332,  0.40945154, -0.09051756, -0.08044092,
        0.11453656, -0.19472562,  0.07121391,  0.3338088 ,  0.17613934,
       -0.3367713 , -0.32266426,  0.21298085,  0.25528452, -0.2730395 ,
       -0.2188933 , -0.07294617, -0.28064996,  0.11466102,  0.23918723],
      dtype=float32)

In [70]:
df.iloc[0,2][:20]

array([-0.22363636,  0.04366332,  0.40945154, -0.09051756, -0.08044092,
        0.11453656, -0.19472562,  0.07121391,  0.3338088 ,  0.17613934,
       -0.3367713 , -0.32266426,  0.21298085,  0.25528452, -0.2730395 ,
       -0.2188933 , -0.07294617, -0.28064996,  0.11466102,  0.23918723],
      dtype=float32)

In [71]:
df.iloc[0,3]

'operation extract leaked report scorecard hughes instant hit buoy blues jack get skate alex chaos maracana build united depleted leicester prevail elliott spoil everton party hungry spurs sense rich picking gunners wide easy target derby raise glass strupar debut double southgate strike leeds pay penalty hammers hand robson youthful lesson saints party wear wolf turned lamb stump mike catch testy gough taunt langer escape hit flintoff injury pile woe england hunters threaten jospin new battle somme kohl successor drawn scandal difference men woman sara denver nurse turned solicitor diana landmine crusade put tories panic yeltsin resignation caught opposition russian roulette sold recovering title hindrance'

In [72]:
df_train=df.iloc[:3623,:] #85% for training and validation, 15% for testing

# Models

In [73]:
from sklearn import  svm, naive_bayes, neighbors, ensemble
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

lr_model = LogisticRegression()
nb_model = naive_bayes.GaussianNB()
knn_model = neighbors.KNeighborsClassifier()
svc_model = svm.SVC(probability=True, gamma="scale")
rf_model = ensemble.RandomForestClassifier(n_estimators=100)
et_model = ensemble.ExtraTreesClassifier(n_estimators=100)
ada_model = ensemble.AdaBoostClassifier()
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, 
                              reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8)

models = ["lr_model", "nb_model", "knn_model", "svc_model", "rf_model", "et_model", "ada_model", "xgb_model"]

In [74]:
def baseline_model_filter(modellist, X, y):
    ''' 1. split the train data further into train and validation (17%). 
        2. fit the train data into each model of the model list
        3. get the classification report based on the model performance on validation data
    '''
    X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size = 0.17, random_state = 100)
    for model_name in modellist:
        curr_model = eval(model_name)
        curr_model.fit(X_train, y_train) 
        print(f'{model_name} \n report:{classification_report(y_valid, curr_model.predict(X_valid))}')

## Using Different word embedding techniques to filter the baseline models

### Bag-Of-Words

In [75]:
print(df_train['news_str'].isnull().sum())

0


In [76]:
count_vect = CountVectorizer(analyzer='word')
X = count_vect.fit_transform(df_train.news_str).toarray()
y = df_train.Label

In [77]:
baseline_model_filter(models, X, y)

C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr_model 
 report:              precision    recall  f1-score   support

           0       0.45      0.44      0.45       284
           1       0.53      0.54      0.54       332

    accuracy                           0.50       616
   macro avg       0.49      0.49      0.49       616
weighted avg       0.49      0.50      0.49       616

nb_model 
 report:              precision    recall  f1-score   support

           0       0.48      0.58      0.53       284
           1       0.57      0.47      0.51       332

    accuracy                           0.52       616
   macro avg       0.53      0.53      0.52       616
weighted avg       0.53      0.52      0.52       616

knn_model 
 report:              precision    recall  f1-score   support

           0       0.50      0.18      0.26       284
           1       0.55      0.84      0.66       332

    accuracy                           0.54       616
   macro avg       0.52      0.51      0.46       616
weighted avg       

## Word level TF-IDF

In [78]:
tfidf_vect = TfidfVectorizer(analyzer='word')
X = tfidf_vect.fit_transform(df_train.news_str).toarray()
y = df_train.Label

In [79]:
baseline_model_filter(models, X, y)

lr_model 
 report:              precision    recall  f1-score   support

           0       0.47      0.30      0.37       284
           1       0.54      0.71      0.62       332

    accuracy                           0.52       616
   macro avg       0.51      0.51      0.49       616
weighted avg       0.51      0.52      0.50       616

nb_model 
 report:              precision    recall  f1-score   support

           0       0.49      0.51      0.50       284
           1       0.56      0.54      0.55       332

    accuracy                           0.53       616
   macro avg       0.52      0.52      0.52       616
weighted avg       0.53      0.53      0.53       616

knn_model 
 report:              precision    recall  f1-score   support

           0       0.47      0.50      0.49       284
           1       0.55      0.52      0.54       332

    accuracy                           0.51       616
   macro avg       0.51      0.51      0.51       616
weighted avg       

## Character Level TF-IDF

In [80]:
tfidf_chars_vect = TfidfVectorizer(analyzer='char')

X = tfidf_chars_vect.fit_transform(df_train.news_str).toarray()
y = df_train.Label

baseline_model_filter(models, X, y)

lr_model 
 report:              precision    recall  f1-score   support

           0       0.62      0.05      0.10       284
           1       0.55      0.97      0.70       332

    accuracy                           0.55       616
   macro avg       0.59      0.51      0.40       616
weighted avg       0.58      0.55      0.42       616

nb_model 
 report:              precision    recall  f1-score   support

           0       0.46      0.99      0.63       284
           1       0.62      0.02      0.03       332

    accuracy                           0.46       616
   macro avg       0.54      0.50      0.33       616
weighted avg       0.55      0.46      0.31       616

knn_model 
 report:              precision    recall  f1-score   support

           0       0.45      0.47      0.46       284
           1       0.53      0.52      0.52       332

    accuracy                           0.49       616
   macro avg       0.49      0.49      0.49       616
weighted avg       

C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


svc_model 
 report:              precision    recall  f1-score   support

           0       0.00      0.00      0.00       284
           1       0.54      1.00      0.70       332

    accuracy                           0.54       616
   macro avg       0.27      0.50      0.35       616
weighted avg       0.29      0.54      0.38       616

rf_model 
 report:              precision    recall  f1-score   support

           0       0.46      0.42      0.44       284
           1       0.54      0.57      0.55       332

    accuracy                           0.50       616
   macro avg       0.50      0.50      0.50       616
weighted avg       0.50      0.50      0.50       616

et_model 
 report:              precision    recall  f1-score   support

           0       0.44      0.38      0.41       284
           1       0.53      0.59      0.56       332

    accuracy                           0.50       616
   macro avg       0.49      0.49      0.48       616
weighted avg       

### Word2vec

In [82]:
X = np.array(list(df_train.doc_vec))
y = np.array(list(df.Label[:3623]))
baseline_model_filter(models, X, y)

C:\Users\harsh\AppData\Local\Temp\ipykernel_11980\3845775326.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y = np.array(list(df.Label[:3623]))


lr_model 
 report:              precision    recall  f1-score   support

           0       0.45      0.33      0.38       284
           1       0.53      0.66      0.59       332

    accuracy                           0.51       616
   macro avg       0.49      0.49      0.49       616
weighted avg       0.50      0.51      0.49       616

nb_model 
 report:              precision    recall  f1-score   support

           0       0.45      0.43      0.44       284
           1       0.53      0.55      0.54       332

    accuracy                           0.50       616
   macro avg       0.49      0.49      0.49       616
weighted avg       0.50      0.50      0.50       616

knn_model 
 report:              precision    recall  f1-score   support

           0       0.53      0.49      0.51       284
           1       0.59      0.62      0.60       332

    accuracy                           0.56       616
   macro avg       0.56      0.56      0.56       616
weighted avg       

### see performance on the test data

In [85]:
df_test = df.iloc[3623:,:]
y_test = df_test.Label
X_test = df_test.news_str

In [92]:
tfidf_chars_vect = TfidfVectorizer(analyzer='word')
train_news = tfidf_chars_vect.fit(df_train.news_str)
X_train_trans = train_news.transform(df_train.news_str).toarray()
X_test_trans = train_news.transform(X_test).toarray()

y_train = df_train.Label

nb_model.fit(X_train_trans, y_train) 


print(f'{nb_model} \n report:{classification_report(y_test, nb_model.predict(X_test_trans))}')

GaussianNB() 
 report:              precision    recall  f1-score   support

           0       0.52      0.47      0.49       229
           1       0.55      0.60      0.58       249

    accuracy                           0.54       478
   macro avg       0.54      0.53      0.53       478
weighted avg       0.54      0.54      0.54       478



In [93]:
with open('label_news_docvec_newsstr.pkl', 'rb') as r:
    df = pickle.load(r)
df.head()

,Label,news,doc_vec,news_str
0,0,"[operation, extract, leaked, report, scorecard...","[-0.22363636, 0.043663323, 0.40945154, -0.0905...",operation extract leaked report scorecard hugh...
1,0,"[scorecard, best, lake, scene, leader, german,...","[-0.30296353, -0.33825478, 0.24209553, -0.0382...",scorecard best lake scene leader german sleaze...
2,0,"[coventry, caught, counter, flo, united, rival...","[-0.3682509, 0.26010576, 0.38839033, 0.1747563...",coventry caught counter flo united rival road ...
3,1,"[pilgrim, know, progress, thatcher, facing, ba...","[-0.3820548, -0.32600626, 0.36784732, -0.03142...",pilgrim know progress thatcher facing ban mcil...
4,1,"[hitches, horlocks, beckham, united, survive, ...","[-0.30124283, -0.22662897, 0.33770743, 0.15407...",hitches horlocks beckham united survive breast...


## Topic Modeling to vectors ( Unsupervised Learning)

#### Data Allocation
1. For Topic Modelling: 15% data set aside for testing. use the 85% for topic modelling
2. Apply the topic model to the testing data to get the topic vectors
3. Create the final train,valid, test files for AWS

Topic Modeling Models
1. Use HDP(Hierarchical Dirichlet Process) to decide topic size
2. Use LDA (Latent Dirichlet Allocation to determine the topics

In [95]:
df_HDP_train = df.news.iloc[:3623]
df_LDA_train = df.news_str.iloc[:3623]
df_LDA_test = df.news_str.iloc[3623:]
print(len(df_LDA_train), len(df_LDA_test))

3623 478


## HDP corpus and dictionary (need to be bag of words format)

In [96]:
id2word_hdp = gensim.corpora.Dictionary(df_HDP_train)
id2word_hdp.filter_extremes(no_below=10, no_above=0.30)
id2word_hdp.compactify()
id2word_hdp.save('train_dict_hdp')
corpus_hdp = [id2word_hdp.doc2bow(doc) for doc in df_HDP_train]

## Use HDP model to decide the maxium topic numbers

In [97]:
from gensim.models import HdpModel
hdp = HdpModel(corpus_hdp, id2word_hdp)

In [98]:
len(hdp.print_topics())

20

In [99]:
hdp.print_topics(num_topics=20)

[(0,
  '0.004*israel + 0.004*china + 0.003*people + 0.003*country + 0.003*war + 0.003*russia + 0.003*one + 0.003*america + 0.003*israeli + 0.003*uk'),
 (1,
  '0.004*review + 0.002*leader + 0.002*football + 0.002*london + 0.002*letters + 0.002*back + 0.002*get + 0.002*city + 0.002*face + 0.002*united'),
 (2,
  '0.005*football + 0.003*england + 0.003*rugby + 0.003*united + 0.003*cricket + 0.002*city + 0.002*cup + 0.002*league + 0.002*union + 0.002*review'),
 (3,
  '0.002*cd + 0.002*review + 0.002*letters + 0.002*leader + 0.001*day + 0.001*football + 0.001*obituary + 0.001*take + 0.001*hit + 0.001*england'),
 (4,
  '0.002*cd + 0.001*back + 0.001*review + 0.001*call + 0.001*cup + 0.001*league + 0.001*take + 0.001*top + 0.001*chelsea + 0.001*obituary'),
 (5,
  '0.002*nan + 0.001*obituary + 0.001*london + 0.001*leader + 0.001*diary + 0.001*letters + 0.001*cd + 0.001*review + 0.001*get + 0.001*make'),
 (6,
  '0.002*england + 0.002*labour + 0.001*review + 0.001*blair + 0.001*part + 0.001*set +

## LDA Modeling for topic vectors

In [105]:
from nltk.corpus import stopwords
from sklearn.feature_extraction import _stop_words
# we need to have more stopwords for topic modeling than for word2vec.
# NLTK + SKlearn + self definded
sk_stop = list(_stop_words.ENGLISH_STOP_WORDS)
mywords = ['whilst', 'say', 'says', 'today','yesterday', 'news', 'tomorrow','iii', 'ii', 'like', 'ha','wa']
final_stop = stopwords.words('english') + mywords + sk_stop

In [106]:
tfv = TfidfVectorizer(stop_words = final_stop, ngram_range = (1, 2), max_df = 0.95)
doc_word = tfv.fit_transform(df_LDA_train).transpose()
corpus = matutils.Sparse2Corpus(doc_word)
id2word = dict((v, k) for k, v in tfv.vocabulary_.items())

In [107]:
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda = models.LdaModel(corpus=corpus, num_topics=20, id2word=id2word, passes=5, random_state = 200 )

AttributeError: 'list' object has no attribute 'LdaModel'